In [1]:
import pandas as pd
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test_data.csv')
print("训练集探索")
print(df_train.head())
print(len(df_train))
print("测试集探索")
print(df_test.head())
print(len(df_test))

训练集探索
   id         1         2         3         4         5         6         7  \
0   1  0.563650  1.069229 -0.837759 -1.122021  0.433296  0.770755 -0.477153   
1   2  0.061333  0.058830  0.056952  0.068634  0.073433  0.072390  0.042975   
2   3  0.035736  0.010964 -0.164872 -0.167714 -0.125075 -0.104771 -0.016650   
3   4 -0.046700  0.060913  0.009340 -0.093400 -0.067817  0.022335  0.006091   
4   5  0.162922 -0.377662  0.014457  0.565437 -0.203369 -0.511508  0.410961   

          8         9  ...        5992      5993      5994      5995  \
0 -0.588421  0.455224  ...   -0.050761  0.220506  0.036548 -0.097461   
1 -0.007302 -0.026286  ...    0.061333  0.107437  0.104516  0.063419   
2  0.151471  0.137258  ...    4.272044 -1.991455 -2.922208  1.937039   
3 -0.076751 -0.032893  ...    0.095025 -0.000406  0.091776  0.074314   
4  0.228546 -0.515244  ...   -0.093563 -0.263632  0.114517  0.209541   

       5996      5997      5998      5999      6000  label  
0 -0.084060 -0.007716 -0.

In [2]:
# 定义DTW距离计算公式，这是利用短路径计算时序数据间距离的一种新方法，但是计算量大
def DTW(s1, s2):
    """
    计算s1和s2之间的距离
    使用曼哈顿距离，如果时间序列输入是二维的那么建议使用欧几里得距离
    """
    from numpy import array, zeros, argmin, inf, equal, ndim
    from sklearn.metrics.pairwise import manhattan_distances
    r, c = len(s1), len(s2)
    D0 = zeros((r+1,c+1))
    D0[0,1:] = inf
    D0[1:,0] = inf
    D1 = D0[1:,1:]
    
    for i in range(r):
        for j in range(c):
            D1[i,j] = manhattan_distances(np.array(s1[i]).reshape(-1,1),np.array(s2[j]).reshape(-1,1))
    M = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i,j] += min(D0[i,j],D0[i,j+1],D0[i+1,j])
    i,j = array(D0.shape) - 2
    
    p,q = [i],[j]
    while(i>0 or j>0):
        tb = argmin((D0[i,j],D0[i,j+1],D0[i+1,j]))
        if tb==0 :
            i-=1
            j-=1
        elif tb==1 :
            i-=1
        else:
            j-=1
        p.insert(0,i)
        q.insert(0,j)
        return D1[-1,-1]

In [3]:
# 测试DTW计算结果
import numpy as np
s1 = np.array([1, 2, 3, 4, 5, 5, 5, 4])
s2 = np.array([3, 4, 5, 5, 5, 4])
print(DTW(s1,s2))

3.0


In [2]:
# 提取训练集的y集
y_train = pd.DataFrame()
y_train['label'] = df_train['label']
df_train.drop(columns='label', inplace=True)

In [3]:
# 定义函数计算时序数据特征值，参考tsfresh，但是减少了特征量
import numpy as np
def get_t(df):
    mean_list = []
    std_list = []
    var_list = []
    median_list = []
    x = pd.DataFrame()
    for i in range(len(df)):
        mean_list.append(np.mean(df.iloc[i][1:].values))
        std_list.append(np.std(df.iloc[i][1:].values))
        var_list.append(np.var(df.iloc[i][1:].values))
        median_list.append(np.median(df.iloc[i][1:].values))
    x['mean'] = mean_list
    x['std'] = std_list
    x['var'] = var_list
    x['median'] = median_list
    return x

In [4]:
# 对训练集和测试集利用时序数据构造新特征
x_train = get_t(df_train)
x_test = get_t(df_test)
print(x_train.head())
print(x_test.head())

       mean       std       var    median
0  0.004765  0.583343  0.340289  0.007919
1  0.015331  0.065739  0.004322  0.017524
2  0.003270  0.540327  0.291954  0.002843
3  0.004026  0.366301  0.134176  0.003249
4  0.013258  0.233977  0.054745  0.010802
       mean       std       var    median
0  0.003489  0.537096  0.288472  0.003249
1  0.006061  0.809820  0.655808 -0.003452
2  0.006581  0.361151  0.130430  0.006497
3  0.009584  0.063453  0.004026  0.011891
4  0.004482  0.152550  0.023271  0.003411


In [23]:
# 使用sklearn中的KNN进行训练
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(x_train, y_train.values.reshape(len(y_train), ))
# 进行结果预测
rst = knn.predict(x_test)
print(rst)
result = pd.DataFrame()
result['id'] = df_test['id']
result['label'] = rst
result.to_csv('data/result.csv', index=False, encoding="utf-8")

[9 7 9 0 1 7 4 9 0 3 7 0 7 0 0 7 8 4 5 0 9 8 7 5 8 9 4 0 1 8 3 7 1 0 9 0 1
 4 6 9 9 3 0 9 9 8 5 6 7 8 1 7 0 7 0 1 9 9 3 5 5 7 1 6 8 6 6 7 1 5 9 1 0 3
 7 9 8 0 0 8 7 1 0 0 1 1 7 4 4 0 9 0 6 7 5 8 0 7 6 0 0 6 6 3 7 3 5 0 2 7 8
 7 0 9 4 9 7 1 0 7 9 2 7 7 9 8 1 7 0 4 5 2 9 7 3 8 0 0 3 0 7 5 4 7 5 0 9 7
 1 4 2 9 9 0 7 3 7 0 4 5 1 0 0 8 0 7 7 6 5 1 9 0 1 4 0 7 3 9 4 8 7 9 1 1 7
 9 8 0 9 7 4 7 0 7 7 1 0 7 9 1 9 0 7 6 7 7 5 0 7 8 9 9 0 9 0 9 4 1 0 9 9 9
 0 7 8 4 6 7 1 3 7 3 9 0 1 5 0 9 9 0 0 1 5 8 8 9 0 5 9 1 0 6 0 9 9 6 7 0 7
 7 4 0 9 0 3 9 3 4 1 5 7 7 7 6 0 9 2 5 1 7 0 5 6 9 9 7 6 3 9 7 0 1 6 4 0 0
 7 9 7 7 7 4 1 3 0 8 0 7 7 8 0 9 1 7 5 9 8 9 6 4 5 8 0 6 9 9 6 6 6 0 0 8 0
 7 7 0 0 0 9 0 0 9 3 7 6 3 1 4 1 7 5 0 6 3 5 9 5 6 1 1 8 6 7 7 9 7 7 7 5 9
 1 7 7 9 8 0 5 1 6 3 0 9 4 7 5 3 1 8 9 6 0 8 5 9 0 1 4 5 8 9 9 8 3 5 7 0 0
 3 7 8 0 9 6 0 1 0 7 4 8 0 9 0 0 1 0 5 9 7 3 8 0 9 7 1 3 9 9 7 6 9 9 7 9 9
 5 0 7 6 8 9 5 0 0 8 7 0 1 8 9 1 8 0 0 1 0 7 4 4 6 7 0 1 4 4 0 5 9 1 7 7 9
 7 4 9 0 5 3 7 7 7 0 5 5 

In [9]:
# 使用SVM进行预测，比对分数，发现表现很差

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
clf = OneVsOneClassifier(LinearSVC(random_state=0))
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_train)
print(confusion_matrix(y_train, rst))

[[177   0   0   0   0   0   0   0   0   0]
 [ 36   7   0   0   0   3   0   0   0   0]
 [ 30   2   0   0   0  11   0   0   0   0]
 [ 47   0   0   0   0   3   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  51]
 [  0   2   0   0   0  45   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  52]
 [  0   0   0   0   0   3   0 100   0  37]
 [ 45   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  34   0 107]]


In [19]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_train)
print(confusion_matrix(y_train, rst))

[[177   0   0   0   0   0   0   0   0   0]
 [  0  46   0   0   0   0   0   0   0   0]
 [  0   0  42   1   0   0   0   0   0   0]
 [  0   0   1  49   0   0   0   0   0   0]
 [  0   0   0   0  51   0   0   0   0   0]
 [  0   0   1   0   0  46   0   0   0   0]
 [  0   0   0   0   0   0  52   0   0   0]
 [  0   0   0   0   0   0   0 140   0   0]
 [  0   0   0   0   0   0   0   0  45   0]
 [  0   0   0   0   0   0   0   0   0 141]]


C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_test)
print(rst)
result = pd.DataFrame()
result['id'] = df_test['id']
result['label'] = rst
result.to_csv('data/result.csv', index=False, encoding="utf-8")

C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[9 7 9 0 1 7 9 9 0 3 9 0 7 0 0 7 8 9 7 0 7 2 7 5 8 9 9 0 1 8 3 7 1 0 9 0 1
 4 6 9 9 3 0 9 9 8 5 9 7 8 1 7 0 7 0 1 9 9 3 5 5 7 1 6 8 6 6 7 1 5 9 1 0 3
 9 9 8 0 0 8 7 1 0 0 1 1 7 4 9 0 9 0 6 7 5 8 0 7 9 0 0 9 6 2 7 3 5 0 2 7 8
 7 0 9 4 9 7 1 0 7 9 2 7 7 9 2 2 7 0 4 2 3 9 7 3 8 0 0 3 0 7 5 4 9 2 0 9 7
 2 4 2 9 7 0 7 2 7 0 4 2 1 0 0 2 0 7 9 6 5 1 9 0 1 4 0 7 3 9 4 8 7 9 1 2 7
 9 8 0 9 7 4 7 0 7 7 5 0 7 7 1 9 0 7 6 9 7 5 0 7 8 9 4 0 9 0 9 4 1 0 9 9 4
 0 7 8 4 6 7 1 3 7 3 9 0 1 2 0 7 7 0 0 1 5 8 8 9 0 2 9 1 0 6 0 9 7 6 7 0 7
 7 4 0 7 0 3 9 3 9 1 2 7 7 7 6 0 9 2 5 2 7 0 5 9 9 9 9 9 3 9 7 0 2 6 4 0 0
 9 9 7 7 7 4 1 3 0 8 0 7 7 8 0 7 1 7 5 9 8 9 6 4 2 8 0 6 9 7 6 6 9 0 0 8 0
 7 7 0 0 0 9 0 0 9 3 7 6 3 1 4 1 7 5 0 6 3 5 9 5 6 1 1 2 9 7 7 9 7 7 7 5 9
 1 9 7 9 2 0 5 1 6 8 0 9 4 7 5 3 1 8 9 6 0 8 5 7 0 1 4 2 8 7 9 8 3 5 7 0 0
 3 9 8 0 9 6 0 1 0 7 4 8 0 9 0 0 1 0 5 9 7 3 2 0 9 7 1 3 9 9 7 6 9 9 7 9 7
 5 0 7 6 8 9 5 0 0 8 7 0 1 8 9 1 8 0 0 1 0 7 4 4 6 7 0 1 4 4 0 5 9 1 7 9 9
 7 9 9 0 5 2 7 7 7 0 5 2 